# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [49]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from IPython import display
import warnings
import pickle

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM MessagesCategories', engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

OperationalError: (sqlite3.OperationalError) no such table: MessagesCategories [SQL: 'SELECT * FROM MessagesCategories']

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Tokenize the text column in the dataset using functions like
    word_tokenize, WordNetLemmatizer and remove the words contained
    in the list of English stopwords
    
    Input
        Text - The text column containing the message information in the dataframe
        
    Output
        List of strings parsed from the text column
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Text Normalization
    
    tokens = word_tokenize(text) # Text Tokenization
    
    lemmatizer = WordNetLemmatizer() # Lemmatize words and remove stop words
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result
    pass

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 40)

In [6]:
%%time
pipeline.fit(X_train, y_train)

CPU times: user 1min 25s, sys: 764 ms, total: 1min 26s
Wall time: 1min 26s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def evaluation_report(actual, predicted, col_names):
    """
    Calculate the evaluation metrics for the model. F1 Score, Precision & Recall
    
    Input
        actual - Array containing the actual labels
        predicted - Array containing the predicted labels
        col_names - List of strings containing names for each of the predicted fields
    
    Output
        metrics_dataframe - Dataframe containing the F1 score, Precision & Recall
                            for the provided actual and predicted fields
    """
    metrics = []
    for i in range(len(col_names)):
        f1 = f1_score(actual[:, i], predicted[:, i], average='micro')
        precision = precision_score(actual[:, i], predicted[:, i], average='micro')
        recall = recall_score(actual[:, i], predicted[:, i], average='micro')
        
        metrics.append([f1, precision, recall])
    
    metrics = np.array(metrics)
    metrics_dataframe = pd.DataFrame(data = metrics, index = col_names, columns = ['F1', 'Precision', 'Recall'])
    
    return metrics_dataframe

In [8]:
col_names = list(y.columns.values)
y_train_pred = pipeline.predict(X_train)

print(evaluation_report(np.array(y_train), y_train_pred, col_names))

                              F1  Precision    Recall
related                 0.989676   0.989676  0.989676
request                 0.986014   0.986014  0.986014
offer                   0.998932   0.998932  0.998932
aid_related             0.984132   0.984132  0.984132
medical_help            0.989065   0.989065  0.989065
medical_products        0.991150   0.991150  0.991150
search_and_rescue       0.994507   0.994507  0.994507
security                0.995575   0.995575  0.995575
military                0.995117   0.995117  0.995117
child_alone             1.000000   1.000000  1.000000
water                   0.993337   0.993337  0.993337
food                    0.994049   0.994049  0.994049
shelter                 0.991964   0.991964  0.991964
clothing                0.997355   0.997355  0.997355
money                   0.995067   0.995067  0.995067
missing_people          0.997101   0.997101  0.997101
refugees                0.993337   0.993337  0.993337
death                   0.99

In [9]:
y_pred = pipeline.predict(X_test)

print(evaluation_report(np.array(y_test), y_pred, col_names))

                              F1  Precision    Recall
related                 0.809735   0.809735  0.809735
request                 0.885261   0.885261  0.885261
offer                   0.995728   0.995728  0.995728
aid_related             0.754654   0.754654  0.754654
medical_help            0.920812   0.920812  0.920812
medical_products        0.948428   0.948428  0.948428
search_and_rescue       0.973146   0.973146  0.973146
security                0.982911   0.982911  0.982911
military                0.970552   0.970552  0.970552
child_alone             1.000000   1.000000  1.000000
water                   0.949802   0.949802  0.949802
food                    0.933628   0.933628  0.933628
shelter                 0.926457   0.926457  0.926457
clothing                0.987641   0.987641  0.987641
money                   0.978334   0.978334  0.978334
missing_people          0.990082   0.990082  0.990082
refugees                0.968416   0.968416  0.968416
death                   0.96

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7feac439f620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [23]:
parameters = {'clf__estimator__min_samples_split': [2, 3]}

cv = GridSearchCV(pipeline, param_grid = parameters)

In [24]:
%%time
cv.fit(X_train, y_train)

CPU times: user 7min 36s, sys: 5.93 s, total: 7min 42s
Wall time: 7min 44s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
%%time
y_pred = cv.predict(X_test)

CPU times: user 6.74 s, sys: 188 ms, total: 6.93 s
Wall time: 6.96 s


In [28]:
print(evaluation_report(np.array(y_test), y_pred, col_names))

                              F1  Precision    Recall
related                 0.807293   0.807293  0.807293
request                 0.881599   0.881599  0.881599
offer                   0.995728   0.995728  0.995728
aid_related             0.752823   0.752823  0.752823
medical_help            0.918370   0.918370  0.918370
medical_products        0.950107   0.950107  0.950107
search_and_rescue       0.972536   0.972536  0.972536
security                0.982911   0.982911  0.982911
military                0.970552   0.970552  0.970552
child_alone             1.000000   1.000000  1.000000
water                   0.953158   0.953158  0.953158
food                    0.925542   0.925542  0.925542
shelter                 0.927373   0.927373  0.927373
clothing                0.987794   0.987794  0.987794
money                   0.978486   0.978486  0.978486
missing_people          0.989625   0.989625  0.989625
refugees                0.967653   0.967653  0.967653
death                   0.95

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [41]:
pipeline2 = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize))
            , ('tfidf', TfidfTransformer())]))
    , ('clf', RandomForestClassifier())])

pipeline2.get_params()

{'memory': None, 'steps': [('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))])),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_start=False))], 'text_pipeline': Pipeline(memory=None,
      steps=

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
%%time
pipeline2.fit(X_train, y_train)

CPU times: user 32.2 s, sys: 691 ms, total: 32.9 s
Wall time: 33 s


Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [44]:
col_names = list(y.columns.values)
y_train_pred = pipeline2.predict(X_train)

print(evaluation_report(np.array(y_train), y_train_pred, col_names))

                              F1  Precision    Recall
related                 0.990032   0.990032  0.990032
request                 0.987590   0.987590  0.987590
offer                   0.998983   0.998983  0.998983
aid_related             0.982097   0.982097  0.982097
medical_help            0.986573   0.986573  0.986573
medical_products        0.991506   0.991506  0.991506
search_and_rescue       0.993236   0.993236  0.993236
security                0.994711   0.994711  0.994711
military                0.995219   0.995219  0.995219
child_alone             1.000000   1.000000  1.000000
water                   0.994456   0.994456  0.994456
food                    0.992676   0.992676  0.992676
shelter                 0.990642   0.990642  0.990642
clothing                0.997610   0.997610  0.997610
money                   0.995321   0.995321  0.995321
missing_people          0.997355   0.997355  0.997355
refugees                0.993337   0.993337  0.993337
death                   0.99

In [46]:
y_pred = pipeline2.predict(X_test)

print(evaluation_report(np.array(y_test), y_pred, col_names))

                              F1  Precision    Recall
related                 0.805005   0.805005  0.805005
request                 0.885719   0.885719  0.885719
offer                   0.995880   0.995880  0.995880
aid_related             0.720781   0.720781  0.720781
medical_help            0.924321   0.924321  0.924321
medical_products        0.951327   0.951327  0.951327
search_and_rescue       0.972688   0.972688  0.972688
security                0.982606   0.982606  0.982606
military                0.969637   0.969637  0.969637
child_alone             1.000000   1.000000  1.000000
water                   0.950412   0.950412  0.950412
food                    0.924321   0.924321  0.924321
shelter                 0.918370   0.918370  0.918370
clothing                0.986878   0.986878  0.986878
money                   0.978181   0.978181  0.978181
missing_people          0.990235   0.990235  0.990235
refugees                0.966433   0.966433  0.966433
death                   0.95

### 9. Export your model as a pickle file

In [51]:
model = pipeline

pickle.dump(model, open('trained_classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.